In [11]:
import pandas as pd

# Load the crowdfunding data
df_crowdfunding = pd.read_excel('Resources/crowdfunding.xlsx')

# Preview the data to understand its structure
print(df_crowdfunding.head())


FileNotFoundError: [Errno 2] No such file or directory: 'Resources/crowdfunding.xlsx'

In [ ]:
# Split "category & sub-category" into separate category and subcategory columns
df_crowdfunding[['category', 'subcategory']] = df_crowdfunding['category & sub-category'].str.split('/', expand=True)

# Get unique categories and subcategories
categories = df_crowdfunding['category'].unique()
subcategories = df_crowdfunding['subcategory'].unique()

# Create category DataFrame
df_category = pd.DataFrame({
    'category_id': ['cat' + str(i+1) for i in range(len(categories))],
    'category': categories
})

# Create subcategory DataFrame
df_subcategory = pd.DataFrame({
    'subcategory_id': ['subcat' + str(i+1) for i in range(len(subcategories))],
    'subcategory': subcategories
})

# Save category and subcategory DataFrames to CSV
df_category.to_csv('category.csv', index=False)
df_subcategory.to_csv('subcategory.csv', index=False)


In [ ]:
# Create the campaign DataFrame with necessary columns
df_campaign = df_crowdfunding[[
    'cf_id', 'contact_id', 'company_name', 'blurb', 'goal', 'pledged', 'outcome', 
    'backers_count', 'country', 'currency', 'launched_at', 'deadline', 'category', 'subcategory'
]]

# Rename columns
df_campaign.rename(columns={
    'blurb': 'description',
    'launched_at': 'launch_date',
    'deadline': 'end_date'
}, inplace=True)

# Convert goal and pledged to float
df_campaign['goal'] = df_campaign['goal'].astype(float)
df_campaign['pledged'] = df_campaign['pledged'].astype(float)

# Convert launch_date and end_date to datetime
df_campaign['launch_date'] = pd.to_datetime(df_campaign['launch_date'], unit='s')
df_campaign['end_date'] = pd.to_datetime(df_campaign['end_date'], unit='s')

# Merge category_id and subcategory_id from category and subcategory DataFrames
df_campaign = df_campaign.merge(df_category, on='category')
df_campaign = df_campaign.merge(df_subcategory, on='subcategory')

# Export the campaign DataFrame to CSV
df_campaign.to_csv('campaign.csv', index=False)


In [ ]:
# Load the contacts data
df_contacts = pd.read_excel('Resources/contacts.xlsx')

# Split the data to extract name and email
df_contacts['name'] = df_contacts['name'].str.split(' ', expand=True)

# Now extract the contact information
contact_data = []
for _, row in df_contacts.iterrows():
    contact_id = row['contact_id']
    first_name, last_name = row['name'].split(' ', 1)
    email = row['email']
    contact_data.append({'contact_id': contact_id, 'first_name': first_name, 'last_name': last_name, 'email': email})

# Create a new DataFrame with the extracted data
df_cleaned_contacts = pd.DataFrame(contact_data)

# Export the contacts DataFrame to CSV
df_cleaned_contacts.to_csv('contacts.csv', index=False)


In [ ]:
CREATE TABLE category (
    category_id VARCHAR PRIMARY KEY,
    category VARCHAR
);

CREATE TABLE subcategory (
    subcategory_id VARCHAR PRIMARY KEY,
    subcategory VARCHAR
);

CREATE TABLE contacts (
    contact_id INT PRIMARY KEY,
    first_name VARCHAR,
    last_name VARCHAR,
    email VARCHAR
);

CREATE TABLE campaign (
    cf_id INT PRIMARY KEY,
    contact_id INT REFERENCES contacts(contact_id),
    company_name VARCHAR,
    description TEXT,
    goal FLOAT,
    pledged FLOAT,
    outcome VARCHAR,
    backers_count INT,
    country VARCHAR,
    currency VARCHAR,
    launch_date TIMESTAMP,
    end_date TIMESTAMP,
    category_id VARCHAR REFERENCES category(category_id),
    subcategory_id VARCHAR REFERENCES subcategory(subcategory_id)
);


In [ ]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="crowdfunding_db", user="your_user", password="your_password", host="localhost"
)
cur = conn.cursor()

# Import each CSV into the respective table
with open('category.csv', 'r') as f:
    next(f)  # Skip the header
    cur.copy_from(f, 'category', sep=',')

with open('subcategory.csv', 'r') as f:
    next(f)
    cur.copy_from(f, 'subcategory', sep=',')

with open('campaign.csv', 'r') as f:
    next(f)
    cur.copy_from(f, 'campaign', sep=',')

with open('contacts.csv', 'r') as f:
    next(f)
    cur.copy_from(f, 'contacts', sep=',')

conn.commit()
cur.close()
conn.close()
